In [1]:
import pandas as pd
import numpy as np
import math
import re
import requests
import bs4
import time

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [3]:
URL = 'https://www.bbcgoodfood.com/search?tab=recipe'

BASIC_URL = 'https://www.bbcgoodfood.com/recipes'

SEARCH_URL = "https://www.bbcgoodfood.com/search"
MEAL_TYPE = "?mealType="

DRIVER = webdriver.Chrome("/home/felipe/Downloads/chromedriver")

/var/folders/sh/41fnk4317z18sxj_vwp76hz40000gn/T/ipykernel_47158/507558712.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  DRIVER = webdriver.Chrome("/home/felipe/Downloads/chromedriver")


In [4]:
def get_category_urls(seacrh_url: str = SEARCH_URL, bbc_food_url: str = URL, search_type: str = MEAL_TYPE) -> pd.DataFrame:
    page = ""
    try:
        page = requests.get(URL)
    except requests.exceptions.ConnectionError:
        print("Connection problems")
        # TODO

    parser = bs4.BeautifulSoup(page.text)
    meal_categories = parser.find('ul', attrs={'role' : "listbox", 'class': 'ma-reset pa-reset'})
    categories = pd.DataFrame(columns=['type', 'url', 'number'])
    for link in meal_categories.find_all('li'):
        val = link.select_one('input')['value']
        categories.loc[len(categories)] = [val,  seacrh_url + search_type + val, int(re.findall(r'\d+', link.getText())[0])]
    categories.to_csv(r"/Users/arina/study/ds/project/food_recommendation/input/bcc_goodfood/categories_links_bbc.csv", sep="\t", index=False)
    return categories

In [5]:
categories = get_category_urls()
categories

,type,url,number
0,afternoon-tea,https://www.bbcgoodfood.com/search?mealType=af...,1266
1,breads,https://www.bbcgoodfood.com/search?mealType=br...,2
2,breakfast,https://www.bbcgoodfood.com/search?mealType=br...,641
3,brunch,https://www.bbcgoodfood.com/search?mealType=br...,541
4,buffet,https://www.bbcgoodfood.com/search?mealType=bu...,1168
5,canapes,https://www.bbcgoodfood.com/search?mealType=ca...,298
6,cheese-course,https://www.bbcgoodfood.com/search?mealType=ch...,10
7,cocktails,https://www.bbcgoodfood.com/search?mealType=co...,242
8,condiment,https://www.bbcgoodfood.com/search?mealType=co...,348
9,dessert,https://www.bbcgoodfood.com/search?mealType=de...,2656


In [6]:
def get_ingredients(ingredients_space):
    ingredients = []
    try:
        ingredients_soup = ingredients_space.find_all('li', attrs={'class': 'pb-xxs pt-xxs list-item list-item--separator'})
        for ingredient in range(len(ingredients_soup)):
            ingredients.append(ingredients_soup[ingredient].getText())
    except AttributeError:
        # TODO find_all - nothing
        ...
    return ingredients

def get_health_banners(health_banners_space):
    health_banners = []
    try:
        health_banners_soup = health_banners_space.find_all('li', attrs={'class': 'list-item'})
        for health_banner in range(len(health_banners_soup)):
            health_banners.append(health_banners_soup[health_banner].getText())
    except AttributeError:
            # TODO find_all - nothing
            ...
    
    return health_banners

In [7]:
def generate_dataset(type: str, url: str, n_items: int, basic_url: str = BASIC_URL, search_url: str = SEARCH_URL, driver: webdriver.Chrome = DRIVER) -> pd.DataFrame:
    result = pd.DataFrame(columns=['type', 'name', 'ingredients', 'difficulty', "health_banners"])

    articles = get_all_articles(url, n_items)

    print(len(articles))
    j = 0
    for article in articles:
        recipe_url = article.find('a').get('href')
        recipe_title = article.find('h2', attrs={'class': 'heading-4'}).getText()
        recipe_request = ""
        while recipe_request == "":
            try:
                recipe_request = requests.get(basic_url + recipe_url)
                break
            except requests.exceptions.ConnectionError:
                # TODO
                print("exc")
                time.sleep(5)
                continue
        recipe_parser = bs4.BeautifulSoup(recipe_request.text)

        ingredients_space = recipe_parser.find('section', attrs={'class': 'recipe__ingredients'})
        planner_space = recipe_parser.find('ul', attrs={'class': 'post-header__planning'})
        health_banners_space = recipe_parser.find('ul', attrs={'class': 'post-header__term-icons-list'})
        try:
            ingredients = get_ingredients(ingredients_space)
            planner_soup = planner_space.find('div', attrs={'class': 'post-header__skill-level'})
            health_banners = get_health_banners(health_banners_space)
            j = j + 1
            result.loc[len(result)] = [type, recipe_title, ingredients, planner_soup.getText(), health_banners]
        except:
            # TODO
            print("find_all None error thing. Ignore this page")
    return result



In [8]:
def get_all_articles(url: str, n_items: int, driver: webdriver.Chrome = DRIVER):
    if n_items > 40:
        n_items = 31
    parsed_articles = ''   
    pages = math.ceil(n_items / 30)

    for n_page in range(1,  pages):
        if n_page < 2:
            current_url = url
        else: 
            current_url = url + "&page=" + str(n_page) 
        print(current_url)
        driver.get(current_url)
        elem = driver.find_element(By.CLASS_NAME, 'load-more-paginator__btn')
        driver.execute_script("arguments[0].click();", elem)
        driver_parser = bs4.BeautifulSoup(driver.page_source)
        list = driver_parser.find('div', attrs={'class': 'layout-md-rail__primary'})
        parsed_articles = driver_parser.find_all('article', attrs={'class': 'card text-align-left card--horizontal card--inline card--with-borders'})
        print(len(parsed_articles))

    # for article in parsed_articles:
    #     recipe_title = article.find('h2', attrs={'class': 'heading-4'}).getText()
    #     print(recipe_title)
    if parsed_articles == '':
        print("A")
    return parsed_articles



In [9]:
def generate_csv():
    data = pd.DataFrame(columns=['type', 'name', 'ingredients', 'difficulty', "health_banners"])
# data = data.append(generate_dataset(categories.iloc[0][ "type"], categories.iloc[0]["url"], categories.iloc[0]["number"]))
# data.to_csv(r"/Users/arina/study/ds/project/food_recommendation/input/bcc_goodfood/data_goodfood_bbc.csv", sep="\t", index=False)

    for i, row in categories.iterrows():
        temp = generate_dataset(row["type"], row["url"], row["number"])
        data = data.append(temp)
        data.to_csv(r"/Users/arina/study/ds/project/food_recommendation/input/bcc_goodfood/data_goodfood_bbc_short.csv", sep="\t", index=False)
generate_csv()

https://www.bbcgoodfood.com/search?mealType=afternoon-tea
30
30


/var/folders/sh/41fnk4317z18sxj_vwp76hz40000gn/T/ipykernel_47158/2149236222.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(temp)
/var/folders/sh/41fnk4317z18sxj_vwp76hz40000gn/T/ipykernel_47158/2149236222.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(temp)


A
0
https://www.bbcgoodfood.com/search?mealType=breakfast
30
30


/var/folders/sh/41fnk4317z18sxj_vwp76hz40000gn/T/ipykernel_47158/2149236222.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(temp)


https://www.bbcgoodfood.com/search?mealType=brunch
30
30


...

/var/folders/sh/41fnk4317z18sxj_vwp76hz40000gn/T/ipykernel_47158/2149236222.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(temp)


https://www.bbcgoodfood.com/search?mealType=treat
30
30


/var/folders/sh/41fnk4317z18sxj_vwp76hz40000gn/T/ipykernel_47158/2149236222.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(temp)


https://www.bbcgoodfood.com/search?mealType=vegetable
30
30
A
0
A
0
A
0


/var/folders/sh/41fnk4317z18sxj_vwp76hz40000gn/T/ipykernel_47158/2149236222.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(temp)
/var/folders/sh/41fnk4317z18sxj_vwp76hz40000gn/T/ipykernel_47158/2149236222.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(temp)
/var/folders/sh/41fnk4317z18sxj_vwp76hz40000gn/T/ipykernel_47158/2149236222.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(temp)
/var/folders/sh/41fnk4317z18sxj_vwp76hz40000gn/T/ipykernel_47158/2149236222.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(temp)


In [10]:
def generate_dataset(type, subtype, url):
        """
            ['type', 'recipe_title', 'ingredients']. 
        """
        final = pd.DataFrame(columns=['type', 'subtype', 'name', 'ingredients'])
        page_count = get_last_page(url)
        for i in range(1, page_count):
            sub_category = url + "?page=" + str(i)
            print(sub_category)
            page = ""
            while page == "":
                try:
                    page = requests.get(sub_category)
                    break
                except requests.exceptions.ConnectionError:
                    print("Connection refused by server...")
                    print("Sleeping for 5 seconds...")
                    time.sleep(5)
                    continue
            soup = BeautifulSoup(page.content, 'lxml')
            grid = soup.find('div', attrs={'class': 'fixed-grid'})
            articles = grid.find_all('article', attrs={'class': 'fixed-recipe-card'})
            j = 0
            for article in articles:
                ingredients = []
                recipe_url = article.find('a').get('href')
                recipe_title = article.find('span', attrs={'class': 'fixed-recipe-card__title-link'}).getText()
                recipe_request = ""
                while recipe_request == "":
                    try:
                        recipe_request = requests.get(recipe_url)
                        break
                    except requests.exceptions.ConnectionError:
                        print("Connection refused by server...")
                        print("Sleeping for 5 seconds...")
                        time.sleep(5)
                        continue
                recipe_soup = BeautifulSoup(recipe_request.content, 'lxml')
                ingredients_space = recipe_soup.find('div', attrs={'id': 'polaris-app'})
                try:
                    ingredients_soup = ingredients_space.find_all('label', attrs={'ng-class': '{true: \'checkList__item\'}[true]'})
    
                    for ingredient in range(len(ingredients_soup)):
                        ingredients.append(ingredients_soup[ingredient]['title'])
    
                    j = j + 1
                    final.loc[len(final)] = [type, subtype, recipe_title, ingredients]
                except:
                    print("find_all None error thing. Ignore this page")
        return final

In [11]:
# parallel

In [12]:
from concurrent.futures import ThreadForestExecutor 

ImportError: cannot import name 'ThreadForestExecutor' from 'concurrent.futures' (/Users/arina/miniconda3/envs/class_1/lib/python3.10/concurrent/futures/__init__.py)

In [ ]:
with ThreadPoolExecutor(max_workers=15) as executor:
    results = executor.map(function, arg_list1)

In [ ]:
# def generate_dataset(type: str, url: str, n_items: int, basic_url: str = BASIC_URL, search_url: str = SEARCH_URL) -> pd.DataFrame:
#     result = pd.DataFrame(columns=['type', 'name', 'ingredients', 'difficulty'])

#     pages = math.ceil(n_items / 30)

#     for n_page in range(1,  pages):
#         if n_page < 2:
#             current_url = url
#         else: 
#             current_url = url + "?page=" + str(n_page) 
#         page = ""
#         print(current_url)
#         while page == "":
#             try:
#                 page = requests.get(current_url)
#                 break
#             except requests.exceptions.ConnectionError:
#                 # TODO
#                 print("e")
#                 time.sleep(5)
#                 continue
#         parser = bs4.BeautifulSoup(page.text)
#         list = parser.find('div', attrs={'class': 'layout-md-rail__primary'})
#         articles = parser.find_all('article', attrs={'class': 'card text-align-left card--horizontal card--inline card--with-borders'})

#         print(len(articles))
#         j = 0
#         for article in articles:
#             ingredients = []
#             recipe_url = article.find('a').get('href')
#             recipe_title = article.find('h2', attrs={'class': 'heading-4'}).getText()
#             print(recipe_title)
#             recipe_request = ""
#             while recipe_request == "":
#                 try:
#                     recipe_request = requests.get(basic_url + recipe_url)
#                     break
#                 except requests.exceptions.ConnectionError:
#                     # TODO
#                     print("exc")
#                     time.sleep(5)
#                     continue
#             recipe_parser = bs4.BeautifulSoup(recipe_request.text)
#             ingredients_space = recipe_parser.find('section', attrs={'class': 'recipe__ingredients'})

#             planner_space = recipe_parser.find('ul', attrs={'class': 'post-header__planning'})
#             try:
#                 ingredients_soup = ingredients_space.find_all('li', attrs={'class': 'pb-xxs pt-xxs list-item list-item--separator'})
#                 for ingredient in range(len(ingredients_soup)):
#                     ingredients.append(ingredients_soup[ingredient].getText())
                
#                 planner_soup = planner_space.find('div', attrs={'class': 'post-header__skill-level'})
#                 j = j + 1
                
#                 result.loc[len(result)] = [type, recipe_title, ingredients, planner_soup.getText()]
#             except:
#                 # TODO
#                 print("find_all None error thing. Ignore this page")
#     return result


In [ ]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from time import sleep

# def get_cpf():
#     driver = webdriver.Chrome("/home/felipe/Downloads/chromedriver")
#     driver.get('https://www.bbcgoodfood.com/search?tab=recipe&mealType=afternoon-tea')
#     elem = driver.find_element(By.CLASS_NAME, 'load-more-paginator__btn')
#     elem1 = driver.execute_script("arguments[0].click();", elem)
#     parser = bs4.BeautifulSoup(driver.page_source)
#     list = parser.find('div', attrs={'class': 'layout-md-rail__primary'})
#     articles = parser.find_all('article', attrs={'class': 'card text-align-left card--horizontal card--inline card--with-borders'})

#     print(len(articles))

#     for article in articles:
#         recipe_title = article.find('h2', attrs={'class': 'heading-4'}).getText()
#         print(recipe_title)

#     driver.get('https://www.bbcgoodfood.com/search?tab=recipe&mealType=afternoon-tea&page=2')
#     elem = driver.find_element(By.CLASS_NAME, 'load-more-paginator__btn')
#     elem1 = driver.execute_script("arguments[0].click();", elem)
#     # print(driver.page_source)
#     parser = bs4.BeautifulSoup(driver.page_source)
#     list = parser.find('div', attrs={'class': 'layout-md-rail__primary'})
#     articles = parser.find_all('article', attrs={'class': 'card text-align-left card--horizontal card--inline card--with-borders'})

#     print(len(articles))

#     for article in articles:
#         recipe_title = article.find('h2', attrs={'class': 'heading-4'}).getText()
#         print(recipe_title)
#     # text=driver.find_elements(By.CLASS_NAME, 'card__content')
#     # print(text)
# get_cpf()

In [ ]:
# def generate_dataset(type: str, url: str, n_items: int, basic_url: str = BASIC_URL, search_url: str = SEARCH_URL, driver: ChromeDriver = DRIVER) -> pd.DataFrame:
#     result = pd.DataFrame(columns=['type', 'name', 'ingredients', 'difficulty'])

#     articles = get_all_articles(url, n_items)
#     print(a)
#     # pages = math.ceil(n_items / 30)

#     # for n_page in range(1,  pages):
#     #     if n_page < 2:
#     #         current_url = url
#     #     else: 
#     #         current_url = url + "?page=" + str(n_page) 
#     #     page = ""
#     #     print(current_url)
#     #     while page == "":
#     #         try:
#     #             page = requests.get(current_url)
#     #             break
#     #         except requests.exceptions.ConnectionError:
#     #             # TODO
#     #             print("e")
#     #             time.sleep(5)
#     #             continue
#     #     parser = bs4.BeautifulSoup(page.text)
#     #     list = parser.find('div', attrs={'class': 'layout-md-rail__primary'})
#         # articles = parser.find_all('article', attrs={'class': 'card text-align-left card--horizontal card--inline card--with-borders'})

#         print(len(articles))
#         j = 0
#         for article in articles:
#             ingredients = []
#             recipe_url = article.find('a').get('href')
#             recipe_title = article.find('h2', attrs={'class': 'heading-4'}).getText()
#             print(recipe_title)
#             recipe_request = ""
#             while recipe_request == "":
#                 try:
#                     recipe_request = requests.get(basic_url + recipe_url)
#                     break
#                 except requests.exceptions.ConnectionError:
#                     # TODO
#                     print("exc")
#                     time.sleep(5)
#                     continue
#             recipe_parser = bs4.BeautifulSoup(recipe_request.text)
#             ingredients_space = recipe_parser.find('section', attrs={'class': 'recipe__ingredients'})

#             planner_space = recipe_parser.find('ul', attrs={'class': 'post-header__planning'})
#             try:
#                 ingredients_soup = ingredients_space.find_all('li', attrs={'class': 'pb-xxs pt-xxs list-item list-item--separator'})
#                 for ingredient in range(len(ingredients_soup)):
#                     ingredients.append(ingredients_soup[ingredient].getText())
                
#                 planner_soup = planner_space.find('div', attrs={'class': 'post-header__skill-level'})
#                 j = j + 1
                
#                 result.loc[len(result)] = [type, recipe_title, ingredients, planner_soup.getText()]
#             except:
#                 # TODO
#                 print("find_all None error thing. Ignore this page")
#     return result
